In [452]:
"""
Created on Tuesday 21 Sep 2023
Author: ZAW
"""
##import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

### Read the dataset

In [407]:
df = pd.read_excel('data/list_of_detected_peak_pairs.xlsx',skiprows=2)

In [408]:
# Column labels
df.columns

Index(['No', 'RT (s)', 'Normalized RT (s)', 'm/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity', 'nCharge', 'nTag', 'Unnamed: 9',
       'NovaMT Library No.', 'External Identifier', 'Compound',
       'Mass Error (ppm)', 'RT Error (s)', 'NovaMT Library No..1',
       'External Identifier.1', 'Compound.1', 'Mass Error (ppm).1',
       'RT Error (s).1', 'Zero-reaction', 'One-reaction', 'Two-reaction',
       'Unnamed: 23', 'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25',

## Data cleaning

#### Extract the relevant columns from the dataframe

In [409]:
columns = ['Normalized RT (s)','m/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity','Compound','Compound.1','DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27',
       'No DR.28', 'No DR.29']
df = df[columns]

In [410]:
#Drop the empty rows(i.e Nan): only drop row if both tier 1 and tier 2 are empty
df.drop(df[(df['Compound'].isna() | (df['Compound'] == '')) & (df['Compound.1'].isna() | (df['Compound.1'] == ''))].index, inplace=True)

In [411]:
#iterate row over tier 1 and repalce NaN with values from tier2 column
for index, row in df.iterrows():
    if pd.isnull(row['Compound']):
        df.at[index, 'Compound'] = row['Compound.1']

In [412]:
# drop the tier 2 column
df = df.drop('Compound.1', axis=1)

In [413]:
df.columns

Index(['Normalized RT (s)', 'm/z_light', 'm/z_heavy', 'Neutral Mass (Da)',
       'Intensity', 'Compound', 'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5',
       'DR.6', 'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11',
       'DR.12', 'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17',
       'DR.18', 'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22',
       'DR.23', 'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27',
       'No DR.28', 'No DR.29'],
      dtype='object')

In [414]:
df

,Normalized RT (s),m/z_light,m/z_heavy,Neutral Mass (Da),Intensity,Compound,DR,DR.1,DR.2,DR.3,...,No DR.20,No DR.21,No DR.22,No DR.23,No DR.24,No DR.25,No DR.26,No DR.27,No DR.28,No DR.29
18,386.3,398.1376,400.1438,150.0524,1449.809490,L-Arabinose,0.8630,0.7320,0.8620,0.7700,...,0.803,0.963,1.082,0.830,0.852,0.861,0.702,0.891,1.031,0.800
19,392.5,307.1225,309.1285,59.0373,1152.039064,Aminoacetaldehyde,1.1760,1.7130,0.8350,1.0260,...,0.438,0.619,0.669,0.790,0.571,0.729,0.585,0.522,0.664,0.746
41,581.4,394.1433,396.1498,146.0580,1150.063645,(R)-4-Dehydropantoic acid,0.8670,0.8920,0.8880,0.9790,...,0.849,0.963,0.942,1.017,0.902,0.917,0.902,0.846,0.999,0.981
46,623.4,394.1433,396.1496,146.0581,1303.804362,3-Hydroxy-5-oxohexanoic acid,0.9450,0.9830,0.9370,0.9890,...,0.885,0.961,0.961,1.044,0.876,0.909,0.995,0.848,0.992,1.006
49,683.3,292.1114,294.1179,44.0262,1113.644708,Acetaldehyde,0.9300,0.9280,0.9610,1.0140,...,0.821,0.944,0.949,1.030,0.861,0.882,0.889,0.764,0.944,0.945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,1034.5,402.1010,404.1073,168.0426,1044.374071,Vanillic acid,0.9866,0.2761,0.8764,0.1797,...,1.133,1.973,1.061,0.602,1.243,0.754,1.723,1.017,1.500,0.780
804,1048.1,372.0905,374.0971,138.0322,1314.406927,4-Hydroxybenzoic acid,0.5708,1.9410,1.0534,1.6530,...,1.364,2.177,1.093,0.911,1.785,0.689,1.439,1.161,0.876,0.750
805,1048.3,307.1129,309.1196,146.1091,5248.849898,Lysine (2 tags 2 charges),0.6710,0.7590,0.4060,0.8890,...,1.779,0.860,1.121,0.761,1.678,0.581,0.882,1.179,0.644,0.949
809,1080.0,307.1479,309.1547,73.0896,1559.591727,Diethylamine,0.7140,1.0360,1.0080,0.6750,...,0.587,0.824,0.964,0.835,0.562,0.651,0.774,0.877,0.985,0.804


In [415]:
#transpose the DataFrame such that the 'Compound' column becomes the column index 
df = df.set_index('Compound').T

In [416]:
df.head(10)

Compound,L-Arabinose,Aminoacetaldehyde,(R)-4-Dehydropantoic acid,3-Hydroxy-5-oxohexanoic acid,Acetaldehyde,3-Hydroxypropanal,Dihydrophloroglucinol,2-Oxovaleric acid,Adipate semialdehyde,Acrolein,...,5-Aminopentanal,Theophylline,"cis,cis-4-Hydroxymuconic semialdehyde",Ornithine (2 tags 2 charges),2-Hydroxymuconate semialdehyde,Vanillic acid,4-Hydroxybenzoic acid,Lysine (2 tags 2 charges),Diethylamine,Histidine
Normalized RT (s),386.30000,392.500000,581.400000,623.400000,683.300000,687.800000,717.000000,731.400000,765.100000,774.800000,...,824.500000,918.600000,921.900000,975.900000,1009.100000,1034.500000,1048.100000,1048.300000,1080.000000,1112.000000
m/z_light,398.13760,307.122500,394.143300,394.143300,292.111400,322.122300,376.132600,364.132500,378.147000,304.111600,...,335.143000,414.123000,376.085600,300.105100,376.085400,402.101000,372.090500,307.112900,307.147900,389.128500
m/z_heavy,400.14380,309.128500,396.149800,396.149600,294.117900,324.128800,378.138700,366.138700,380.154800,306.120100,...,337.149400,416.129400,378.091600,302.111900,378.091800,404.107300,374.097100,309.119600,309.154700,391.134900
Neutral Mass (Da),150.05240,59.037300,146.058000,146.058100,44.026200,74.037100,128.047400,116.047300,130.061800,56.026400,...,101.084700,180.064700,142.027300,132.093400,142.027100,168.042600,138.032200,146.109100,73.089600,155.070100
Intensity,1449.80949,1152.039064,1150.063645,1303.804362,1113.644708,1735.581226,1021.290947,1113.653816,1047.512093,1334.650335,...,21583.925319,1139.529581,1151.586552,5584.712865,1087.320634,1044.374071,1314.406927,5248.849898,1559.591727,5920.234434
DR,0.86300,1.176000,0.867000,0.945000,0.930000,0.947000,1.557000,1.083000,0.996000,0.437000,...,1.005000,0.955100,1.224900,0.477000,0.657900,0.986600,0.570800,0.671000,0.714000,0.612000
DR.1,0.73200,1.713000,0.892000,0.983000,0.928000,0.908000,1.292000,0.979000,0.968000,0.458000,...,0.994000,0.992000,0.151300,0.497000,1.084900,0.276100,1.941000,0.759000,1.036000,0.446000
DR.2,0.86200,0.835000,0.888000,0.937000,0.961000,0.952000,1.347000,1.083000,1.005000,0.876000,...,0.329000,1.030500,0.582900,0.283500,0.917900,0.876400,1.053400,0.406000,1.008000,0.098800
DR.3,0.77000,1.026000,0.979000,0.989000,1.014000,1.040000,1.749000,0.983000,1.006000,0.626000,...,1.110000,0.771800,0.920000,1.142000,0.850000,0.179700,1.653000,0.889000,0.675000,0.781000
DR.4,0.81200,0.848000,0.956000,0.960000,0.973000,0.968000,1.768000,0.955000,0.994000,0.472000,...,1.408000,0.977000,1.253100,0.612000,0.894200,0.227500,1.606000,1.247000,1.263000,0.731000


In [417]:
# Drop the top 5 row
df = df.drop(df.index[:5])

In [418]:
#Rename the 'Compound' column to 'SampleID'
df = df.rename_axis('SampleID', axis='columns')

In [419]:
# add class column for classification
df['Class'] = df.index.map(lambda x: 'DR' if x.startswith('DR') else 'No DR')


In [420]:
# Rseset index
df.reset_index(drop=True, inplace=True)

In [421]:
df.head(10)

SampleID,L-Arabinose,Aminoacetaldehyde,(R)-4-Dehydropantoic acid,3-Hydroxy-5-oxohexanoic acid,Acetaldehyde,3-Hydroxypropanal,Dihydrophloroglucinol,2-Oxovaleric acid,Adipate semialdehyde,Acrolein,...,Theophylline,"cis,cis-4-Hydroxymuconic semialdehyde",Ornithine (2 tags 2 charges),2-Hydroxymuconate semialdehyde,Vanillic acid,4-Hydroxybenzoic acid,Lysine (2 tags 2 charges),Diethylamine,Histidine,Class
0,0.863,1.176,0.867,0.945,0.930,0.947,1.557,1.083,0.996,0.437,...,0.9551,1.2249,0.4770,0.6579,0.9866,0.5708,0.671,0.7140,0.6120,DR
1,0.732,1.713,0.892,0.983,0.928,0.908,1.292,0.979,0.968,0.458,...,0.9920,0.1513,0.4970,1.0849,0.2761,1.9410,0.759,1.0360,0.4460,DR
2,0.862,0.835,0.888,0.937,0.961,0.952,1.347,1.083,1.005,0.876,...,1.0305,0.5829,0.2835,0.9179,0.8764,1.0534,0.406,1.0080,0.0988,DR
3,0.770,1.026,0.979,0.989,1.014,1.040,1.749,0.983,1.006,0.626,...,0.7718,0.9200,1.1420,0.8500,0.1797,1.6530,0.889,0.6750,0.7810,DR
4,0.812,0.848,0.956,0.960,0.973,0.968,1.768,0.955,0.994,0.472,...,0.9770,1.2531,0.6120,0.8942,0.2275,1.6060,1.247,1.2630,0.7310,DR
5,1.013,0.809,0.951,1.058,0.980,1.034,1.868,0.972,1.020,0.772,...,1.5270,1.5673,0.6320,1.3220,1.2623,0.7303,0.363,0.6041,0.3540,DR
6,0.537,0.983,0.876,0.864,0.826,0.816,1.265,0.706,0.867,0.403,...,1.6670,2.1380,0.5570,1.8035,1.7220,0.9963,0.528,1.3800,0.2420,DR
7,1.022,1.599,0.920,0.975,0.968,0.970,1.661,0.997,0.988,0.415,...,1.2600,1.1951,0.3940,0.8528,0.9626,1.6832,0.448,0.7100,0.3690,DR
8,1.490,0.603,0.902,0.953,0.928,0.932,1.635,0.920,0.971,0.811,...,1.4690,0.1315,0.6220,1.1780,1.0639,1.8470,1.005,0.6650,1.4300,No DR
9,1.315,0.758,0.974,0.977,0.948,0.936,1.676,0.930,1.035,0.837,...,0.7890,0.1326,0.4080,1.2530,0.2418,1.0410,1.102,0.6270,0.7090,DR


In [426]:
# Drop duplicate columns
df = df.loc[:,~df.columns.duplicated()]

#### Transform the Data

In [435]:
x = df.iloc[:, :167]
y = df.iloc[:,-1]


In [437]:
## Convert the categorical variables to number
LabelEncoder_gender = LabelEncoder()
y = LabelEncoder_gender.fit_transform(y)

### PLS-DA for classificaiton

In [454]:
# Set up the PLS-DA classifier
model = PLSRegression(n_components=2)
lda = LinearDiscriminantAnalysis()

# Set up StratifiedKFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform permutation test
score, permutation_scores, pvalue = permutation_test_score(
    model, x, y, scoring="roc_auc", cv=cv, n_permutations=100, n_jobs=1)

print(f"Model score: {score:.2f}")
print(f"Permutation test p-value: {pvalue:.4f}")

Model score: 0.82
Permutation test p-value: 0.0099


### XGB for classificaiton

In [440]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")

# Set up StratifiedKFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform permutation test
score, permutation_scores, pvalue = permutation_test_score(
    model, x, y, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)

print(f"Model score: {score:.2f}")
print(f"Permutation test p-value: {pvalue:.4f}")

Model score: 0.76
Permutation test p-value: 0.0099


### GB for classificaiton

In [448]:
# Set up the Gradient Boosting classifier
model = GradientBoostingClassifier()

# Set up StratifiedKFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform permutation test
score, permutation_scores, pvalue = permutation_test_score(
    model, x, y, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)

print(f"Model score: {score:.2f}")
print(f"Permutation test p-value: {pvalue:.4f}")

Model score: 0.67
Permutation test p-value: 0.0495


### RF for classificaiton

In [446]:
# Set up the Random Forest classifier
model = RandomForestClassifier()

# Set up StratifiedKFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform permutation test
score, permutation_scores, pvalue = permutation_test_score(
    model, x, y, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)

print(f"Model score: {score:.2f}")
print(f"Permutation test p-value: {pvalue:.4f}")

Model score: 0.82
Permutation test p-value: 0.0099


### SVM for classificaiton

In [455]:
# Set up the SVM classifier
model = SVC()

# Set up StratifiedKFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform permutation test
score, permutation_scores, pvalue = permutation_test_score(
    model, x, y, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)

print(f"Model score: {score:.2f}")
print(f"Permutation test p-value: {pvalue:.4f}")

Model score: 0.69
Permutation test p-value: 0.0198
